In [1]:
from selenium import webdriver
import time
import csv

import pandas as pd
from PIL import Image
import tempfile
import os
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException  
import shutil

    

/Users/shogonakazaato/.pyenv/versions/3.7.4/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
driver_path="/Users/okamotochiaki/PycharmProjects/fiby2/chromedriver"

In [3]:
def get_driver():
    return webdriver.Chrome(driver_path)

def scraper(driver, category, lastpage, parameter = None):
    if parameter is None:
        parameter = []
    base_url = "https://baseconnect.in/companies/category"
    time_sleep =2

    kanji_url_lists=[]
    url_lists=[]
    company_name_lists=[]
    phase_1_failure_page_list=[]
    phase_2_failure_page_list=[]

    for page in range(1, lastpage + 1):
        query = '&'.join(parameter)
        kanji_url= "{base_url}/{category}?page={page}&{query}".format(base_url=base_url, category=category, page=page, query=query)
        kanji_url_lists.append(kanji_url)

    for kanji_url in kanji_url_lists:
        try:
            driver.get(kanji_url)
            time.sleep(time_sleep)
            search=[]
            search=driver.find_elements_by_xpath('//h4/a')

            for ii in search:
                try:
                    company_name=ii.get_attribute('.')
                    company_page_link=ii.get_attribute('href')
                    url_lists.append(company_page_link)
                    company_name_lists.append(company_name)
                except NoSuchElementException:
                    phase_2_failure_page_list.append(kanji_url)
        except NoSuchElementException:
            phase_1_failure_page_list.append(kanji_url)

            #     except NoSuchElementException:
            failure_page_list.append(kanji_url)
            time.sleep(time_sleep)

    hp_lists=[]
    tag_lists=[]
    target_lists=[]
    phase_3_failure_page_lists=[]
    company_name_lists=[]
    count = 0
    for link in url_lists:
        try:
            time.sleep(time_sleep)
            contact_url = get_detail_page(driver, link)
            if contact_url is not None:
                hp_lists.append(get_detail_page(driver, link))

            #         try:
    #             industry=driver.find_element_by_xpath('//div[1]/div[1]/div[3]/ul/li/a')
    #             industry_tag=company_hp.get_attribute('.')
    #             tag_lists.append(industry_tag)
    #         except:
    #             target_list.append(company_hp_link)

        except NoSuchElementException:
            phase_3_failure_page_lists.append(link)

    return pd.DataFrame({'url': hp_lists})


def get_detail_page(driver, url):
    try:
        driver.get(url)
        company_hp=driver.find_element_by_css_selector('.node__contact p:last-child a')
        return company_hp.get_attribute('href')

    except NoSuchElementException:
        print("以下のページでエラーが起きました.\n{}".format(url))
        return None

In [4]:
#　手順
# title部分に好きな名前(基本、業界名)
# category部分にbase connectのcategory以降「?」以前のの文字列
# 例えば https://baseconnect.in/companies/category/a2b5e24f-c628-44e5-a341-b6e4a6e4f0e3?employee_number_gte=50&employee_number_lte=500
# だったらa2b5e24f-c628-44e5-a341-b6e4a6e4f0e3
# last_pageにその業界の最終ページ
# parameterで検索条件を指定する

targets = [
    {
        "title": "IT業界の会社(Webサービス・アプリ運営業界の会社)",
        "category": "b1d5c1e6-7cc5-41c4-9552-28530d2c9e9c",
        "last_page": 12,
        "parameter": ["employee_number_gte=50", "employee_number_lte=500", "prefecture%5B%5D=a30ebdff-361b-42be-8372-1b0d350f48a0"],
    },
    {
        "title": "IT業界の会社(クラウド・フィンテック業界の会社)",
        "category": "578d6793-48f8-4776-906f-756a0b42f195",
        "last_page": 4,
        "parameter": ["employee_number_gte=50", "employee_number_lte=500", "prefecture%5B%5D=a30ebdff-361b-42be-8372-1b0d350f48a0"],
    },
    {
        "title": "IT業界の会社(デジタルコンテンツ業界の会社)",
        "category": "37b7583c-431d-408d-ac66-fd1a18f84c41",
        "last_page": 3,
        "parameter": ["employee_number_gte=50",  "employee_number_lte=500", "prefecture%5B%5D=a30ebdff-361b-42be-8372-1b0d350f48a0"],
    },
    {
        "title": "広告業界の会社(インターネット広告代理店業界の会社)",
        "category": "56705ce0-0fe4-40cb-9659-68f5eaa45d27",
        "last_page": 4,
        "parameter": ["employee_number_gte=50",  "employee_number_lte=500", "prefecture%5B%5D=a30ebdff-361b-42be-8372-1b0d350f48a0"],
    },
    {
        "title": "ゲーム業界の会社(ソーシャルゲーム業界の会社)",
        "category": "5e227ff5-308f-4cd8-955c-b5174189a70a",
        "last_page": 3,
        "parameter": ["employee_number_gte=50",  "employee_number_lte=500", "prefecture%5B%5D=a30ebdff-361b-42be-8372-1b0d350f48a0"],
    },
    {
        "title": "ゲーム業界の会社(ゲームソフト開発業界の会社)",
        "category": "6d8b55ed-3e28-49b7-8c05-587833d73ca9",
        "last_page": 2,
        "parameter": ["employee_number_gte=50",  "employee_number_lte=500", "prefecture%5B%5D=a30ebdff-361b-42be-8372-1b0d350f48a0"],
    },
    {
        "title": "コンサルティング業界の会社(Webマーケティング業界の会社)",
        "category": "c6b6da03-db2e-49c0-a5b3-a3923339d4d8",
        "last_page": 4,
        "parameter": ["employee_number_gte=50",  "employee_number_lte=500", "prefecture%5B%5D=a30ebdff-361b-42be-8372-1b0d350f48a0"],
    },
]

driver = get_driver()
to_csv_df = pd.DataFrame()
for target in targets:
    df = scraper(driver, target['category'], target['last_page'], parameter=target['parameter'])
    df['title'] = target['title']
    to_csv_df = pd.concat([to_csv_df, df])

to_csv_df.to_csv("it_url.csv")

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=85.0.4183.102)


In [ ]:
!pwd